In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import joblib

from sklearn.metrics import (
    precision_recall_fscore_support,
    accuracy_score,
    balanced_accuracy_score,
    f1_score,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.utils import shuffle

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

print("✅ All imports are working correctly!")


✅ All imports are working correctly!


In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import lightgbm as lgb
import joblib

from sklearn.metrics import (
    precision_recall_fscore_support,
    accuracy_score,
    balanced_accuracy_score,
    f1_score,
    confusion_matrix,
)
from sklearn.utils import shuffle

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE


# ============================================================
# 0) Paths and basic settings
# ============================================================

DATA_PATH = Path(r"C:\Users\LENOVO\Downloads\DF_ALL_4MOD_TRIMMED.csv")
MODELS_DIR = Path(r"C:\Users\LENOVO\Downloads\sp-data pre\LightGBM\output")

os.makedirs(MODELS_DIR, exist_ok=True)

SUBJECT_COL = "subject"
LABEL_COL   = "label"

RANDOM_STATE = 42
FOLD_SIZE_SUBJECTS = 5  # 5-subject grouped CV (نفس فكرة RF)

np.random.seed(RANDOM_STATE)


# ============================================================
# 1) Metadata columns (NOT used as features)
# ============================================================

meta_cols = [
    "subject",
    "run",
    "window_idx",
    "label",
    "run_base",
    "ecg_start_time_sec",
    "t_start",
    "t_end",
    "win_idx",
    "Unnamed: 0",
    "timestamp_center",
]


# ============================================================
# 2) Helper functions
# ============================================================

def hybrid_resample_l012(
    X,
    y,
    majority_class=0,
    mid_class=1,
    minority_class=2,
    max_majority_ratio=2.0,
    random_state=42,
):
    """
    Hybrid sampling strategy for labels {0, 1, 2} using:
      - Random UNDER-sampling for class 0 (majority)
      - SMOTE over-sampling for class 1 (preictal) and class 2 (seizure)
    """
    rng = np.random.RandomState(random_state)
    X = np.asarray(X)
    y = np.asarray(y)

    idx_maj = np.where(y == majority_class)[0]
    idx_mid = np.where(y == mid_class)[0]
    idx_min = np.where(y == minority_class)[0]

    n_maj = len(idx_maj)
    n_mid = len(idx_mid)
    n_min = len(idx_min)

    if n_mid == 0 or n_min == 0:
        raise ValueError("Classes 1 and 2 must be present for hybrid_resample_l012.")

    # 1) target counts
    mid_oversample_factor = 1.5
    target_mid = max(n_mid, int(n_mid * mid_oversample_factor))

    minority_target_ratio = 0.7
    raw_target_min = int(target_mid * minority_target_ratio)
    max_minority_factor = 10.0
    max_min = int(n_min * max_minority_factor)
    target_min = max(n_min, min(raw_target_min, max_min))

    target_maj = min(n_maj, int(max_majority_ratio * target_mid))

    sampling_strategy_under = {
        majority_class: target_maj,
        mid_class: n_mid,
        minority_class: n_min,
    }

    rus = RandomUnderSampler(
        sampling_strategy=sampling_strategy_under,
        random_state=random_state,
    )
    X_under, y_under = rus.fit_resample(X, y)

    n_mid_under = np.sum(y_under == mid_class)
    n_min_under = np.sum(y_under == minority_class)

    sampling_strategy_smote = {
        mid_class: max(n_mid_under, target_mid),
        minority_class: max(n_min_under, target_min),
    }

    try:
        smote = SMOTE(
            sampling_strategy=sampling_strategy_smote,
            random_state=random_state,
            k_neighbors=1,
        )
        X_bal, y_bal = smote.fit_resample(X_under, y_under)
    except ValueError:
        print("[WARN] SMOTE failed, falling back to simple oversampling.")

        X_bal = X_under
        y_bal = y_under

        idx_maj_u = np.where(y_under == majority_class)[0]
        idx_mid_u = np.where(y_under == mid_class)[0]
        idx_min_u = np.where(y_under == minority_class)[0]

        n_mid_u = len(idx_mid_u)
        n_min_u = len(idx_min_u)

        if n_mid_u < target_mid:
            extra_mid = target_mid - n_mid_u
            extra_idx_mid = rng.choice(idx_mid_u, size=extra_mid, replace=True)
            X_bal = np.concatenate([X_bal, X_under[extra_idx_mid]], axis=0)
            y_bal = np.concatenate([y_bal, y_under[extra_idx_mid]], axis=0)

        if n_min_u < target_min:
            extra_min = target_min - n_min_u
            extra_idx_min = rng.choice(idx_min_u, size=extra_min, replace=True)
            X_bal = np.concatenate([X_bal, X_under[extra_idx_min]], axis=0)
            y_bal = np.concatenate([y_bal, y_under[extra_idx_min]], axis=0)

    idx_all = np.arange(len(y_bal))
    idx_all = shuffle(idx_all, random_state=random_state)

    X_res = X_bal[idx_all]
    y_res = y_bal[idx_all]

    return X_res, y_res


def build_lgbm(params, num_classes):
    """
    Create a LightGBM classifier (multiclass, GPU).
    """
    return lgb.LGBMClassifier(
        objective="multiclass",
        num_class=num_classes,
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        num_leaves=params["num_leaves"],
        learning_rate=params["learning_rate"],
        subsample=params["subsample"],
        colsample_bytree=params["colsample_bytree"],
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=-1,
        device_type="gpu",
        tree_learner="data",
        gpu_platform_id=0,
        gpu_device_id=0,
        max_bin=255,
    )


def lgbm_fit_predict(X_train, y_train, X_test, params, num_classes):
    model = build_lgbm(params, num_classes)
    print("Device type:", model.get_params()["device_type"])
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    return model, y_test_pred


def evaluate_metrics(y_true, y_pred, label_set=None):
    if label_set is None:
        label_set = np.unique(y_true)

    prec, rec, f1, support = precision_recall_fscore_support(
        y_true, y_pred, labels=label_set, zero_division=0
    )

    cm = confusion_matrix(y_true, y_pred, labels=label_set)
    total = cm.sum()

    tp = []
    fp = []
    fn = []
    tn = []

    for i in range(len(label_set)):
        TP = cm[i, i]
        FP = cm[:, i].sum() - TP
        FN = cm[i, :].sum() - TP
        TN = total - (TP + FP + FN)
        tp.append(TP)
        fp.append(FP)
        fn.append(FN)
        tn.append(TN)

    metrics = {
        "labels": label_set,
        "precision_per_class": prec,
        "recall_per_class": rec,
        "f1_per_class": f1,
        "support_per_class": support,
        "tp_per_class": np.array(tp),
        "fp_per_class": np.array(fp),
        "fn_per_class": np.array(fn),
        "tn_per_class": np.array(tn),
        "accuracy": accuracy_score(y_true, y_pred),
        "balanced_accuracy": balanced_accuracy_score(y_true, y_pred),
        "micro_f1": f1_score(y_true, y_pred, average="micro"),
        "macro_f1": f1_score(y_true, y_pred, average="macro"),
        "weighted_f1": f1_score(y_true, y_pred, average="weighted"),
        "confusion_matrix": cm,
    }

    return metrics


def print_metrics(metrics, header=""):
    labels  = metrics["labels"]
    prec    = metrics["precision_per_class"]
    rec     = metrics["recall_per_class"]
    f1      = metrics["f1_per_class"]
    support = metrics["support_per_class"]
    cm      = metrics["confusion_matrix"]

    if header:
        print("\n" + "=" * 70)
        print(header)
        print("=" * 70)

    print("\nPer-class metrics:")
    print("label\tprecision\trecall\t\tf1-score\tsupport")
    for i, c in enumerate(labels):
        print(f"{c}\t{prec[i]:.4f}\t\t{rec[i]:.4f}\t\t{f1[i]:.4f}\t\t{support[i]}")

    print("\nGlobal metrics:")
    print(f"Accuracy          : {metrics['accuracy']:.4f}")
    print(f"Balanced Accuracy : {metrics['balanced_accuracy']:.4f}")
    print(f"Micro-F1          : {metrics['micro_f1']:.4f}")
    print(f"Macro-F1          : {metrics['macro_f1']:.4f}")
    print(f"Weighted-F1       : {metrics['weighted_f1']:.4f}")

    print("\nConfusion matrix (rows = true, cols = predicted):")
    print(cm)

    total = cm.sum()
    print("\nPer-class confusion details (TP, FP, FN, TN):")
    for idx, c in enumerate(labels):
        TP = cm[idx, idx]
        FP = cm[:, idx].sum() - TP
        FN = cm[idx, :].sum() - TP
        TN = total - (TP + FP + FN)
        print(f"Label {c}: TP={TP}, FP={FP}, FN={FN}, TN={TN}")


# ============================================================
# 3) Load data
# ============================================================

print("Loading fused dataset...")
df = pd.read_csv(DATA_PATH)

if SUBJECT_COL not in df.columns or LABEL_COL not in df.columns:
    raise ValueError("Check SUBJECT_COL and LABEL_COL names. They are not found in the dataframe.")

print(f"Data shape: {df.shape}")
print("\nLabel distribution:")
print(df[LABEL_COL].value_counts())
print("\nLabel proportions (%):")
print(df[LABEL_COL].value_counts(normalize=True) * 100)

print("\nMetadata columns dropped from training:")
print(meta_cols)

feature_cols = [c for c in df.columns if c not in meta_cols]
print(f"\nNumber of feature columns used for training: {len(feature_cols)}")

X_all = df[feature_cols].astype("float32").values
y_all = df[LABEL_COL].values
subjects_all = df[SUBJECT_COL].values

label_set = np.unique(y_all)
print(f"Unique labels: {label_set}")


# ============================================================
# 4) FIXED hyperparameters for LightGBM (no nested tuning)
# ============================================================

FIXED_PARAMS = {
    "n_estimators": 400,
    "max_depth": 18,
    "num_leaves": 60,
    "learning_rate": 0.05,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
}

print("\nUsing FIXED LightGBM hyperparameters:")
print(FIXED_PARAMS)


# ============================================================
# 5) 5-subject grouped CV (NO nested)
# ============================================================

unique_subjects = np.array(sorted(np.unique(subjects_all)))
outer_folds = [
    unique_subjects[i:i + FOLD_SIZE_SUBJECTS]
    for i in range(0, len(unique_subjects), FOLD_SIZE_SUBJECTS)
]

print(f"\nTotal subjects: {len(unique_subjects)}, "
      f"Number of folds (5-subject CV): {len(outer_folds)}")

all_test_y_true = []
all_test_y_pred = []
outer_macro_f1_list = []

for outer_idx, test_subj in enumerate(outer_folds, start=1):
    print("\n" + "#" * 80)
    print(f"OUTER Fold {outer_idx}/{len(outer_folds)} (5-subject grouped CV)")
    print("#" * 80)

    is_test = np.isin(subjects_all, test_subj)
    is_train = ~is_test

    X_train = X_all[is_train]
    y_train = y_all[is_train]
    X_test  = X_all[is_test]
    y_test  = y_all[is_test]

    print("Train subjects:", len(np.unique(subjects_all[is_train])))
    print("Test subjects :", len(np.unique(test_subj)))

    # Hybrid resampling على train فقط
    X_train_bal, y_train_bal = hybrid_resample_l012(
        X_train, y_train,
        majority_class=0,
        mid_class=1,
        minority_class=2,
        max_majority_ratio=2.0,
        random_state=RANDOM_STATE + outer_idx
    )

    model, y_pred = lgbm_fit_predict(
        X_train_bal, y_train_bal,
        X_test,
        FIXED_PARAMS,
        num_classes=len(label_set)
    )

    model_path = MODELS_DIR / f"lgbm_outer_{outer_idx:02d}_5subj.pkl"
    joblib.dump(model, model_path)
    print(f"Saved model: {model_path}")

    m = evaluate_metrics(y_test, y_pred, label_set)
    print_metrics(m, header=f"OUTER Fold {outer_idx} Test Metrics (LightGBM, GPU, 5-subject CV)")

    all_test_y_true.append(y_test)
    all_test_y_pred.append(y_pred)
    outer_macro_f1_list.append(m["macro_f1"])


# ============================================================
# 6) GLOBAL metrics
# ============================================================

all_test_y_true = np.concatenate(all_test_y_true)
all_test_y_pred = np.concatenate(all_test_y_pred)

global_m = evaluate_metrics(all_test_y_true, all_test_y_pred, label_set)
print_metrics(global_m, header="GLOBAL Metrics Across All 5-subject CV Folds (LightGBM, GPU)")


# ============================================================
# 7) FINAL deployment model (train on ALL subjects with FIXED_PARAMS)
# ============================================================

X_full_bal, y_full_bal = hybrid_resample_l012(
    X_all, y_all,
    majority_class=0,
    mid_class=1,
    minority_class=2,
    max_majority_ratio=2.0,
    random_state=RANDOM_STATE + 999
)

final_model = build_lgbm(FIXED_PARAMS, num_classes=len(label_set))
final_model.fit(X_full_bal, y_full_bal)

deployment_model_path = MODELS_DIR / "lgbm_final_deployment_5subj_gpu.pkl"
joblib.dump(final_model, deployment_model_path)

print("\nFinal LightGBM deployment model (GPU) trained on all subjects and saved to:")
print(deployment_model_path)
print("Use this model for real-time or external testing on new patients.")


Loading fused dataset...
Data shape: (1622677, 113)

Label distribution:
label
0    1312916
1     289844
2      19917
Name: count, dtype: int64

Label proportions (%):
label
0    80.910495
1    17.862088
2     1.227416
Name: proportion, dtype: float64

Metadata columns dropped from training:
['subject', 'run', 'window_idx', 'label', 'run_base', 'ecg_start_time_sec', 't_start', 't_end', 'win_idx', 'Unnamed: 0', 'timestamp_center']

Number of feature columns used for training: 103
Unique labels: [0 1 2]

Using FIXED LightGBM hyperparameters:
{'n_estimators': 400, 'max_depth': 18, 'num_leaves': 60, 'learning_rate': 0.05, 'subsample': 0.8, 'colsample_bytree': 0.8}

Total subjects: 113, Number of folds (5-subject CV): 23

################################################################################
OUTER Fold 1/23 (5-subject grouped CV)
################################################################################
Train subjects: 108
Test subjects : 5
[WARN] SMOTE failed, falling back 

c:\Users\LENOVO\Downloads\sp-data pre\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Saved model: C:\Users\LENOVO\Downloads\sp-data pre\LightGBM\output\lgbm_outer_01_5subj.pkl

OUTER Fold 1 Test Metrics (LightGBM, GPU, 5-subject CV)

Per-class metrics:
label	precision	recall		f1-score	support
0	0.8448		0.9769		0.9061		64940
1	0.3885		0.0370		0.0676		11160
2	0.2343		0.2245		0.2293		1332

Global metrics:
Accuracy          : 0.8285
Balanced Accuracy : 0.4128
Micro-F1          : 0.8285
Macro-F1          : 0.4010
Weighted-F1       : 0.7736

Confusion matrix (rows = true, cols = predicted):
[[63440   631   869]
 [10639   413   108]
 [ 1014    19   299]]

Per-class confusion details (TP, FP, FN, TN):
Label 0: TP=63440, FP=11653, FN=1500, TN=839
Label 1: TP=413, FP=650, FN=10747, TN=65622
Label 2: TP=299, FP=977, FN=1033, TN=75123

################################################################################
OUTER Fold 2/23 (5-subject grouped CV)
################################################################################
Train subjects: 108
Test subjects : 5
[WARN] SMO

c:\Users\LENOVO\Downloads\sp-data pre\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Saved model: C:\Users\LENOVO\Downloads\sp-data pre\LightGBM\output\lgbm_outer_02_5subj.pkl

OUTER Fold 2 Test Metrics (LightGBM, GPU, 5-subject CV)

Per-class metrics:
label	precision	recall		f1-score	support
0	0.8406		0.9455		0.8900		17789
1	0.0600		0.0127		0.0210		3150
2	0.1369		0.3491		0.1967		169

Global metrics:
Accuracy          : 0.8015
Balanced Accuracy : 0.4358
Micro-F1          : 0.8015
Macro-F1          : 0.3692
Weighted-F1       : 0.7547

Confusion matrix (rows = true, cols = predicted):
[[16820   627   342]
 [ 3080    40    30]
 [  110     0    59]]

Per-class confusion details (TP, FP, FN, TN):
Label 0: TP=16820, FP=3190, FN=969, TN=129
Label 1: TP=40, FP=627, FN=3110, TN=17331
Label 2: TP=59, FP=372, FN=110, TN=20567

################################################################################
OUTER Fold 3/23 (5-subject grouped CV)
################################################################################
Train subjects: 108
Test subjects : 5
[WARN] SMOTE faile

...

Saved model: C:\Users\LENOVO\Downloads\sp-data pre\LightGBM\output\lgbm_outer_21_5subj.pkl

OUTER Fold 21 Test Metrics (LightGBM, GPU, 5-subject CV)

Per-class metrics:
label	precision	recall		f1-score	support
0	0.8575		0.9710		0.9107		82501
1	0.3348		0.0507		0.0880		13238
2	0.1784		0.2216		0.1977		1313

Global metrics:
Accuracy          : 0.8353
Balanced Accuracy : 0.4144
Micro-F1          : 0.8353
Macro-F1          : 0.3988
Weighted-F1       : 0.7889

Confusion matrix (rows = true, cols = predicted):
[[80107  1315  1079]
 [12306   671   261]
 [ 1004    18   291]]

Per-class confusion details (TP, FP, FN, TN):
Label 0: TP=80107, FP=13310, FN=2394, TN=1241
Label 1: TP=671, FP=1333, FN=12567, TN=82481
Label 2: TP=291, FP=1340, FN=1022, TN=94399

################################################################################
OUTER Fold 22/23 (5-subject grouped CV)
################################################################################
Train subjects: 108
Test subjects : 5
[WARN

c:\Users\LENOVO\Downloads\sp-data pre\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Saved model: C:\Users\LENOVO\Downloads\sp-data pre\LightGBM\output\lgbm_outer_22_5subj.pkl

OUTER Fold 22 Test Metrics (LightGBM, GPU, 5-subject CV)

Per-class metrics:
label	precision	recall		f1-score	support
0	0.7600		0.9545		0.8462		54218
1	0.0810		0.0037		0.0070		16097
2	0.1239		0.3213		0.1788		940

Global metrics:
Accuracy          : 0.7313
Balanced Accuracy : 0.4265
Micro-F1          : 0.7313
Macro-F1          : 0.3440
Weighted-F1       : 0.6478

Confusion matrix (rows = true, cols = predicted):
[[51750   665  1803]
 [15705    59   333]
 [  634     4   302]]

Per-class confusion details (TP, FP, FN, TN):
Label 0: TP=51750, FP=16339, FN=2468, TN=698
Label 1: TP=59, FP=669, FN=16038, TN=54489
Label 2: TP=302, FP=2136, FN=638, TN=68179

################################################################################
OUTER Fold 23/23 (5-subject grouped CV)
################################################################################
Train subjects: 110
Test subjects : 3
[WARN] SMO

c:\Users\LENOVO\Downloads\sp-data pre\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Saved model: C:\Users\LENOVO\Downloads\sp-data pre\LightGBM\output\lgbm_outer_23_5subj.pkl

OUTER Fold 23 Test Metrics (LightGBM, GPU, 5-subject CV)

Per-class metrics:
label	precision	recall		f1-score	support
0	0.6791		0.8363		0.7495		4312
1	0.6421		0.2177		0.3251		1879
2	0.4144		0.5560		0.4749		714

Global metrics:
Accuracy          : 0.6390
Balanced Accuracy : 0.5367
Micro-F1          : 0.6390
Macro-F1          : 0.5165
Weighted-F1       : 0.6056

Confusion matrix (rows = true, cols = predicted):
[[3606  217  489]
 [1398  409   72]
 [ 306   11  397]]

Per-class confusion details (TP, FP, FN, TN):
Label 0: TP=3606, FP=1704, FN=706, TN=889
Label 1: TP=409, FP=228, FN=1470, TN=4798
Label 2: TP=397, FP=561, FN=317, TN=5630

GLOBAL Metrics Across All 5-subject CV Folds (LightGBM, GPU)

Per-class metrics:
label	precision	recall		f1-score	support
0	0.8122		0.9384		0.8708		1312916
1	0.2034		0.0294		0.0514		289844
2	0.0964		0.3085		0.1468		19917

Global metrics:
Accuracy          : 0.7683
Ba

In [3]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import lightgbm as lgb
import joblib

from sklearn.metrics import (
    precision_recall_fscore_support,
    accuracy_score,
    balanced_accuracy_score,
    f1_score,
    confusion_matrix,
)
from sklearn.utils import shuffle

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE


# ============================================================
# 0) Paths and basic settings
# ============================================================

DATA_PATH = Path(r"C:\Users\LENOVO\Downloads\DF_ALL_4MOD_TRIMMED.csv")
MODELS_DIR = Path(r"C:\Users\LENOVO\Downloads\sp-data pre\LightGBM_only_egg\output")

os.makedirs(MODELS_DIR, exist_ok=True)

SUBJECT_COL = "subject"
LABEL_COL   = "label"

RANDOM_STATE = 42
FOLD_SIZE_SUBJECTS = 5  # 5-subject grouped CV (same idea as RF)

np.random.seed(RANDOM_STATE)


# ============================================================
# 1) Metadata columns (NOT used as features)
# ============================================================

meta_cols = [
    "subject",
    "run",
    "window_idx",
    "label",
    "run_base",
    "ecg_start_time_sec",
    "t_start",
    "t_end",
    "win_idx",
    "Unnamed: 0",
    "timestamp_center",
]


# ============================================================
# 2) Helper functions
# ============================================================

def hybrid_resample_l012(
    X,
    y,
    majority_class=0,
    mid_class=1,
    minority_class=2,
    max_majority_ratio=2.0,
    random_state=42,
):
    """
    Hybrid sampling strategy for labels {0, 1, 2} using:
      - Random UNDER-sampling for class 0 (majority)
      - SMOTE over-sampling for class 1 (preictal) and class 2 (seizure)
    """
    rng = np.random.RandomState(random_state)
    X = np.asarray(X)
    y = np.asarray(y)

    idx_maj = np.where(y == majority_class)[0]
    idx_mid = np.where(y == mid_class)[0]
    idx_min = np.where(y == minority_class)[0]

    n_maj = len(idx_maj)
    n_mid = len(idx_mid)
    n_min = len(idx_min)

    if n_mid == 0 or n_min == 0:
        raise ValueError("Classes 1 and 2 must be present for hybrid_resample_l012.")

    # 1) target counts
    mid_oversample_factor = 1.5
    target_mid = max(n_mid, int(n_mid * mid_oversample_factor))

    minority_target_ratio = 0.7
    raw_target_min = int(target_mid * minority_target_ratio)
    max_minority_factor = 10.0
    max_min = int(n_min * max_minority_factor)
    target_min = max(n_min, min(raw_target_min, max_min))

    target_maj = min(n_maj, int(max_majority_ratio * target_mid))

    sampling_strategy_under = {
        majority_class: target_maj,
        mid_class: n_mid,
        minority_class: n_min,
    }

    rus = RandomUnderSampler(
        sampling_strategy=sampling_strategy_under,
        random_state=random_state,
    )
    X_under, y_under = rus.fit_resample(X, y)

    n_mid_under = np.sum(y_under == mid_class)
    n_min_under = np.sum(y_under == minority_class)

    sampling_strategy_smote = {
        mid_class: max(n_mid_under, target_mid),
        minority_class: max(n_min_under, target_min),
    }

    try:
        smote = SMOTE(
            sampling_strategy=sampling_strategy_smote,
            random_state=random_state,
            k_neighbors=1,
        )
        X_bal, y_bal = smote.fit_resample(X_under, y_under)
    except ValueError:
        print("[WARN] SMOTE failed, falling back to simple oversampling.")

        X_bal = X_under
        y_bal = y_under

        idx_maj_u = np.where(y_under == majority_class)[0]
        idx_mid_u = np.where(y_under == mid_class)[0]
        idx_min_u = np.where(y_under == minority_class)[0]

        n_mid_u = len(idx_mid_u)
        n_min_u = len(idx_min_u)

        if n_mid_u < target_mid:
            extra_mid = target_mid - n_mid_u
            extra_idx_mid = rng.choice(idx_mid_u, size=extra_mid, replace=True)
            X_bal = np.concatenate([X_bal, X_under[extra_idx_mid]], axis=0)
            y_bal = np.concatenate([y_bal, y_under[extra_idx_mid]], axis=0)

        if n_min_u < target_min:
            extra_min = target_min - n_min_u
            extra_idx_min = rng.choice(idx_min_u, size=extra_min, replace=True)
            X_bal = np.concatenate([X_bal, X_under[extra_idx_min]], axis=0)
            y_bal = np.concatenate([y_bal, y_under[extra_idx_min]], axis=0)

    idx_all = np.arange(len(y_bal))
    idx_all = shuffle(idx_all, random_state=random_state)

    X_res = X_bal[idx_all]
    y_res = y_bal[idx_all]

    return X_res, y_res


def build_lgbm(params, num_classes):
    """
    Create a LightGBM classifier (multiclass, GPU).
    """
    return lgb.LGBMClassifier(
        objective="multiclass",
        num_class=num_classes,
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        num_leaves=params["num_leaves"],
        learning_rate=params["learning_rate"],
        subsample=params["subsample"],
        colsample_bytree=params["colsample_bytree"],
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=-1,
        device_type="gpu",
        tree_learner="data",
        gpu_platform_id=0,
        gpu_device_id=0,
        max_bin=255,
    )


def lgbm_fit_predict(X_train, y_train, X_test, params, num_classes):
    model = build_lgbm(params, num_classes)
    print("Device type:", model.get_params()["device_type"])
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    return model, y_test_pred


def evaluate_metrics(y_true, y_pred, label_set=None):
    if label_set is None:
        label_set = np.unique(y_true)

    prec, rec, f1, support = precision_recall_fscore_support(
        y_true, y_pred, labels=label_set, zero_division=0
    )

    cm = confusion_matrix(y_true, y_pred, labels=label_set)
    total = cm.sum()

    tp = []
    fp = []
    fn = []
    tn = []

    for i in range(len(label_set)):
        TP = cm[i, i]
        FP = cm[:, i].sum() - TP
        FN = cm[i, :].sum() - TP
        TN = total - (TP + FP + FN)
        tp.append(TP)
        fp.append(FP)
        fn.append(FN)
        tn.append(TN)

    metrics = {
        "labels": label_set,
        "precision_per_class": prec,
        "recall_per_class": rec,
        "f1_per_class": f1,
        "support_per_class": support,
        "tp_per_class": np.array(tp),
        "fp_per_class": np.array(fp),
        "fn_per_class": np.array(fn),
        "tn_per_class": np.array(tn),
        "accuracy": accuracy_score(y_true, y_pred),
        "balanced_accuracy": balanced_accuracy_score(y_true, y_pred),
        "micro_f1": f1_score(y_true, y_pred, average="micro"),
        "macro_f1": f1_score(y_true, y_pred, average="macro"),
        "weighted_f1": f1_score(y_true, y_pred, average="weighted"),
        "confusion_matrix": cm,
    }

    return metrics


def print_metrics(metrics, header=""):
    labels  = metrics["labels"]
    prec    = metrics["precision_per_class"]
    rec     = metrics["recall_per_class"]
    f1      = metrics["f1_per_class"]
    support = metrics["support_per_class"]
    cm      = metrics["confusion_matrix"]

    if header:
        print("\n" + "=" * 70)
        print(header)
        print("=" * 70)

    print("\nPer-class metrics:")
    print("label\tprecision\trecall\t\tf1-score\tsupport")
    for i, c in enumerate(labels):
        print(f"{c}\t{prec[i]:.4f}\t\t{rec[i]:.4f}\t\t{f1[i]:.4f}\t\t{support[i]}")

    print("\nGlobal metrics:")
    print(f"Accuracy          : {metrics['accuracy']:.4f}")
    print(f"Balanced Accuracy : {metrics['balanced_accuracy']:.4f}")
    print(f"Micro-F1          : {metrics['micro_f1']:.4f}")
    print(f"Macro-F1          : {metrics['macro_f1']:.4f}")
    print(f"Weighted-F1       : {metrics['weighted_f1']:.4f}")

    print("\nConfusion matrix (rows = true, cols = predicted):")
    print(cm)

    total = cm.sum()
    print("\nPer-class confusion details (TP, FP, FN, TN):")
    for idx, c in enumerate(labels):
        TP = cm[idx, idx]
        FP = cm[:, idx].sum() - TP
        FN = cm[idx, :].sum() - TP
        TN = total - (TP + FP + FN)
        print(f"Label {c}: TP={TP}, FP={FP}, FN={FN}, TN={TN}")


# ============================================================
# 3) Load data
# ============================================================

print("Loading fused dataset...")
df = pd.read_csv(DATA_PATH)

if SUBJECT_COL not in df.columns or LABEL_COL not in df.columns:
    raise ValueError("Check SUBJECT_COL and LABEL_COL names. They are not found in the dataframe.")

print(f"Data shape: {df.shape}")
print("\nLabel distribution:")
print(df[LABEL_COL].value_counts())
print("\nLabel proportions (%):")
print(df[LABEL_COL].value_counts(normalize=True) * 100)

print("\nMetadata columns dropped from training:")
print(meta_cols)

# === EEG-only feature columns (channel 1 & channel 2) ===
eeg_feature_cols = [
    "ch1_Delta Bandpower",
    "ch1_Theta Bandpower",
    "ch1_Alpha Bandpower",
    "ch1_Beta Bandpower",
    "ch1_Gamma Bandpower",
    "ch1_Relative Delta Bandpower",
    "ch1_Relative Theta Bandpower",
    "ch1_Relative Alpha Bandpower",
    "ch1_Relative Beta Bandpower",
    "ch1_Relative Gamma Bandpower",
    "ch1_Interquartile Range",
    "ch1_Median",
    "ch1_Entropy",
    "ch1_Skewness",
    "ch1_Kurtosis",
    "ch1_Line Length",
    "ch1_Hjorth Mobility",
    "ch1_Hjorth Complexity",
    "ch2_Delta Bandpower",
    "ch2_Theta Bandpower",
    "ch2_Alpha Bandpower",
    "ch2_Beta Bandpower",
    "ch2_Gamma Bandpower",
    "ch2_Relative Delta Bandpower",
    "ch2_Relative Theta Bandpower",
    "ch2_Relative Alpha Bandpower",
    "ch2_Relative Beta Bandpower",
    "ch2_Relative Gamma Bandpower",
    "ch2_Interquartile Range",
    "ch2_Median",
    "ch2_Entropy",
    "ch2_Skewness",
    "ch2_Kurtosis",
    "ch2_Line Length",
    "ch2_Hjorth Mobility",
    "ch2_Hjorth Complexity",
]

# Optional safety check: ensure all selected feature columns exist in the dataframe
missing_feats = [c for c in eeg_feature_cols if c not in df.columns]
if len(missing_feats) > 0:
    raise ValueError(f"The following EEG feature columns are missing from the dataframe: {missing_feats}")

feature_cols = eeg_feature_cols
print(f"\nNumber of feature columns used for training: {len(feature_cols)}")

X_all = df[feature_cols].astype("float32").values
y_all = df[LABEL_COL].values
subjects_all = df[SUBJECT_COL].values

label_set = np.unique(y_all)
print(f"Unique labels: {label_set}")


# ============================================================
# 4) FIXED hyperparameters for LightGBM (no nested tuning)
# ============================================================

FIXED_PARAMS = {
    "n_estimators": 400,
    "max_depth": 18,
    "num_leaves": 60,
    "learning_rate": 0.05,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
}

print("\nUsing FIXED LightGBM hyperparameters:")
print(FIXED_PARAMS)


# ============================================================
# 5) 5-subject grouped CV (NO nested)
# ============================================================

unique_subjects = np.array(sorted(np.unique(subjects_all)))
outer_folds = [
    unique_subjects[i:i + FOLD_SIZE_SUBJECTS]
    for i in range(0, len(unique_subjects), FOLD_SIZE_SUBJECTS)
]

print(f"\nTotal subjects: {len(unique_subjects)}, "
      f"Number of folds (5-subject CV): {len(outer_folds)}")

all_test_y_true = []
all_test_y_pred = []
outer_macro_f1_list = []

for outer_idx, test_subj in enumerate(outer_folds, start=1):
    print("\n" + "#" * 80)
    print(f"OUTER Fold {outer_idx}/{len(outer_folds)} (5-subject grouped CV)")
    print("#" * 80)

    is_test = np.isin(subjects_all, test_subj)
    is_train = ~is_test

    X_train = X_all[is_train]
    y_train = y_all[is_train]
    X_test  = X_all[is_test]
    y_test  = y_all[is_test]

    print("Train subjects:", len(np.unique(subjects_all[is_train])))
    print("Test subjects :", len(np.unique(test_subj)))

    # Hybrid resampling on TRAIN only
    X_train_bal, y_train_bal = hybrid_resample_l012(
        X_train, y_train,
        majority_class=0,
        mid_class=1,
        minority_class=2,
        max_majority_ratio=2.0,
        random_state=RANDOM_STATE + outer_idx
    )

    model, y_pred = lgbm_fit_predict(
        X_train_bal, y_train_bal,
        X_test,
        FIXED_PARAMS,
        num_classes=len(label_set)
    )

    model_path = MODELS_DIR / f"lgbm_outer_{outer_idx:02d}_5subj.pkl"
    joblib.dump(model, model_path)
    print(f"Saved model: {model_path}")

    m = evaluate_metrics(y_test, y_pred, label_set)
    print_metrics(m, header=f"OUTER Fold {outer_idx} Test Metrics (LightGBM, GPU, 5-subject CV)")

    all_test_y_true.append(y_test)
    all_test_y_pred.append(y_pred)
    outer_macro_f1_list.append(m["macro_f1"])


# ============================================================
# 6) GLOBAL metrics
# ============================================================

all_test_y_true = np.concatenate(all_test_y_true)
all_test_y_pred = np.concatenate(all_test_y_pred)

global_m = evaluate_metrics(all_test_y_true, all_test_y_pred, label_set)
print_metrics(global_m, header="GLOBAL Metrics Across All 5-subject CV Folds (LightGBM, GPU)")


# ============================================================
# 7) FINAL deployment model (train on ALL subjects with FIXED_PARAMS)
# ============================================================

X_full_bal, y_full_bal = hybrid_resample_l012(
    X_all, y_all,
    majority_class=0,
    mid_class=1,
    minority_class=2,
    max_majority_ratio=2.0,
    random_state=RANDOM_STATE + 999
)

final_model = build_lgbm(FIXED_PARAMS, num_classes=len(label_set))
final_model.fit(X_full_bal, y_full_bal)

deployment_model_path = MODELS_DIR / "lgbm_final_deployment_5subj_gpu.pkl"
joblib.dump(final_model, deployment_model_path)

print("\nFinal LightGBM deployment model (GPU) trained on all subjects and saved to:")
print(deployment_model_path)
print("Use this model for real-time or external testing on new patients.")

Loading fused dataset...
Data shape: (1622677, 113)

Label distribution:
label
0    1312916
1     289844
2      19917
Name: count, dtype: int64

Label proportions (%):
label
0    80.910495
1    17.862088
2     1.227416
Name: proportion, dtype: float64

Metadata columns dropped from training:
['subject', 'run', 'window_idx', 'label', 'run_base', 'ecg_start_time_sec', 't_start', 't_end', 'win_idx', 'Unnamed: 0', 'timestamp_center']

Number of feature columns used for training: 36
Unique labels: [0 1 2]

Using FIXED LightGBM hyperparameters:
{'n_estimators': 400, 'max_depth': 18, 'num_leaves': 60, 'learning_rate': 0.05, 'subsample': 0.8, 'colsample_bytree': 0.8}

Total subjects: 113, Number of folds (5-subject CV): 23

################################################################################
OUTER Fold 1/23 (5-subject grouped CV)
################################################################################
Train subjects: 108
Test subjects : 5
Device type: gpu


c:\Users\LENOVO\Downloads\sp-data pre\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Saved model: C:\Users\LENOVO\Downloads\sp-data pre\LightGBM_only_egg\output\lgbm_outer_01_5subj.pkl

OUTER Fold 1 Test Metrics (LightGBM, GPU, 5-subject CV)

Per-class metrics:
label	precision	recall		f1-score	support
0	0.8373		0.9558		0.8926		64940
1	0.0756		0.0138		0.0233		11160
2	0.1203		0.1141		0.1171		1332

Global metrics:
Accuracy          : 0.8056
Balanced Accuracy : 0.3612
Micro-F1          : 0.8056
Macro-F1          : 0.3444
Weighted-F1       : 0.7540

Confusion matrix (rows = true, cols = predicted):
[[62070  1857  1013]
 [10907   154    99]
 [ 1153    27   152]]

Per-class confusion details (TP, FP, FN, TN):
Label 0: TP=62070, FP=12060, FN=2870, TN=432
Label 1: TP=154, FP=1884, FN=11006, TN=64388
Label 2: TP=152, FP=1112, FN=1180, TN=74988

################################################################################
OUTER Fold 2/23 (5-subject grouped CV)
################################################################################
Train subjects: 108
Test subjects : 5

c:\Users\LENOVO\Downloads\sp-data pre\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Saved model: C:\Users\LENOVO\Downloads\sp-data pre\LightGBM_only_egg\output\lgbm_outer_02_5subj.pkl

OUTER Fold 2 Test Metrics (LightGBM, GPU, 5-subject CV)

Per-class metrics:
label	precision	recall		f1-score	support
0	0.8438		0.9494		0.8935		17789
1	0.1136		0.0206		0.0349		3150
2	0.1073		0.3314		0.1621		169

Global metrics:
Accuracy          : 0.8058
Balanced Accuracy : 0.4338
Micro-F1          : 0.8058
Macro-F1          : 0.3635
Weighted-F1       : 0.7595

Confusion matrix (rows = true, cols = predicted):
[[16888   507   394]
 [ 3013    65    72]
 [  113     0    56]]

Per-class confusion details (TP, FP, FN, TN):
Label 0: TP=16888, FP=3126, FN=901, TN=193
Label 1: TP=65, FP=507, FN=3085, TN=17451
Label 2: TP=56, FP=466, FN=113, TN=20473

################################################################################
OUTER Fold 3/23 (5-subject grouped CV)
################################################################################
Train subjects: 108
Test subjects : 5
Device ty

...

c:\Users\LENOVO\Downloads\sp-data pre\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Saved model: C:\Users\LENOVO\Downloads\sp-data pre\LightGBM_only_egg\output\lgbm_outer_08_5subj.pkl

OUTER Fold 8 Test Metrics (LightGBM, GPU, 5-subject CV)

Per-class metrics:
label	precision	recall		f1-score	support
0	0.7845		0.9423		0.8562		71070
1	0.2952		0.0245		0.0452		18993
2	0.0448		0.3071		0.0782		534

Global metrics:
Accuracy          : 0.7461
Balanced Accuracy : 0.4246
Micro-F1          : 0.7461
Macro-F1          : 0.3265
Weighted-F1       : 0.6816

Confusion matrix (rows = true, cols = predicted):
[[66966  1105  2999]
 [18032   465   496]
 [  365     5   164]]

Per-class confusion details (TP, FP, FN, TN):
Label 0: TP=66966, FP=18397, FN=4104, TN=1130
Label 1: TP=465, FP=1110, FN=18528, TN=70494
Label 2: TP=164, FP=3495, FN=370, TN=86568

################################################################################
OUTER Fold 9/23 (5-subject grouped CV)
################################################################################
Train subjects: 108
Test subjects : 5


c:\Users\LENOVO\Downloads\sp-data pre\xgb_gpu_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Saved model: C:\Users\LENOVO\Downloads\sp-data pre\LightGBM_only_egg\output\lgbm_outer_09_5subj.pkl

OUTER Fold 9 Test Metrics (LightGBM, GPU, 5-subject CV)

Per-class metrics:
label	precision	recall		f1-score	support
0	0.8064		0.8565		0.8307		42237
1	0.1504		0.0269		0.0457		9879
2	0.0159		0.3151		0.0303		292

Global metrics:
Accuracy          : 0.6971
Balanced Accuracy : 0.3995
Micro-F1          : 0.6971
Macro-F1          : 0.3022
Weighted-F1       : 0.6783

Confusion matrix (rows = true, cols = predicted):
[[36178  1501  4558]
 [ 8486   266  1127]
 [  198     2    92]]

Per-class confusion details (TP, FP, FN, TN):
Label 0: TP=36178, FP=8684, FN=6059, TN=1487
Label 1: TP=266, FP=1503, FN=9613, TN=41026
Label 2: TP=92, FP=5685, FN=200, TN=46431

################################################################################
OUTER Fold 10/23 (5-subject grouped CV)
################################################################################
Train subjects: 108
Test subjects : 5
Dev

KeyboardInterrupt: 